In [1]:
using PyCall
np = pyimport("numpy")
using DifferentialEquations
using DynamicalSystems
using PyPlot
#using DiffEqDevTools
#using ProbNumDiffEq
#using OrdinaryDiffEq
#using DiffEqCallbacks
#using DataFrames
using ForwardDiff
#using Latexify, LaTeXStrings, SymEngine

In [2]:
include("Utils\\BS_Sin_System.jl")
include("Utils\\BS_Cos_System.jl")
include("Utils\\BS_Uni_System.jl")
include("Utils\\BS_Exp_System.jl")

BS_Exp_Sys (generic function with 1 method)

In [3]:
pushfirst!(PyVector(pyimport("sys")."path"), "")
BS =  pyimport("Bick_Systems")

PyObject <module 'Bick_Systems' from 'C:\\Users\\artyo\\YandexDisk\\Other\\Python\\coupled_contours_2\\Bick_Systems.py'>

In [4]:
@inline @inbounds function BS_Py_Sys(u, p, t)
    K, a2, a4, r, Eps = p
    rhs  = BS.coupled_Bick(K, a2, a4, r, Eps)
    du = rhs(t, u)
    return SVector{6}(du)
end

BS_Py_Sys (generic function with 1 method)

# Бенчмарки

In [16]:
function Funcs_Compare(first_func, second_func)
    max = 0
    vecs = np.random.normal(size=(10000, 6), loc=[np.pi]*6, scale=10)
    
    for i = 1 : size(vecs)[1] - 1
        diff = np.max(np.abs([first_func(vecs[i, :]) - second_func(vecs[i, :])]))
        if diff > max
           max = diff
        end
    end
    return max
end

function Time_benchmark(func, num)
    vecs = np.random.normal(size=(num, 6), loc=[np.pi]*6, scale=0.001)
    @time for i = 1 : size(vecs)[1] - 1
        func(vecs[i, :])
    end
end

Time_benchmark (generic function with 1 method)

# Параметры системы

In [6]:
K = 0.4
Eps = 1e-5
a2 = pi/2
a4 = pi
r = 0.1
#startPt = [3.141592653589786; 2.0776989018145287e-09; 6.283185307057817;
#           3.1415926535897944; 6.2831853071795605; 3.1415916495384404]

val = 0.0001
#startPt = [3.141592653589786 + val, 2.0776989018145287e-09 + val, 6.283185307057817 + val,
#            3.1415926535897944 + val, 6.2831853071795605 + val, 3.1415916495384404 + val]
startPt = [np.pi - val; 0 + val; 0 + val;
           np.pi - val; 0 + val; np.pi - val]

6-element Vector{Float64}:
 3.141492653589793
 0.0001
 0.0001
 3.141492653589793
 0.0001
 3.141492653589793

### Сравнение точностей и времени вычисления различных реализаций системы

In [10]:
Funcs_Compare(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_sin)(0, x),
              x -> BS_Uni_Sys(couple_sin)(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

0

In [27]:
Time_benchmark(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_sin)(0, x), 10000)

  1.194651 seconds (828.74 k allocations: 32.672 MiB, 3.04% gc time, 0.38% compilation time)


In [28]:
Time_benchmark(x -> BS_Uni_Sys(couple_sin)(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.065101 seconds (385.44 k allocations: 28.926 MiB, 23.88% compilation time)


---

In [13]:
Funcs_Compare(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_sin)(0, x),
              x -> BS_Sin_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

1.5745326049920738e-15

In [29]:
Time_benchmark(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_sin)(0, x), 10000)

  1.122061 seconds (828.74 k allocations: 32.672 MiB, 0.47% compilation time)


In [30]:
Time_benchmark(x -> BS_Sin_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.053292 seconds (205.39 k allocations: 9.699 MiB, 30.12% compilation time)


---

In [14]:
Funcs_Compare(x -> BS_Sin_Sys_Simp(zeros(6, 1), x, [K, a2, a4, r, Eps], 0),
              x -> BS_Sin_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

1.1176769145609944e-16

In [31]:
Time_benchmark(x -> BS_Sin_Sys_Simp(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.136820 seconds (205.39 k allocations: 9.699 MiB, 35.08% gc time, 10.42% compilation time)


In [32]:
Time_benchmark(x -> BS_Sin_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.051782 seconds (205.39 k allocations: 9.699 MiB, 28.82% compilation time)


---

In [11]:
Funcs_Compare(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_cos)(0, x),
              x -> BS_Uni_Sys(couple_cos)(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

0

In [33]:
Time_benchmark(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_cos)(0, x), 10000)

  1.146423 seconds (828.74 k allocations: 32.672 MiB, 0.45% compilation time)


In [34]:
Time_benchmark(x -> BS_Uni_Sys(couple_cos)(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.066796 seconds (385.44 k allocations: 28.926 MiB, 20.95% compilation time)


---

In [12]:
Funcs_Compare(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_cos)(0, x),
              x -> BS_Cos_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

1.5743462577436779e-15

In [35]:
Time_benchmark(x -> BS.coupled_Bick(K, a2, a4, r, Eps, BS.couple_cos)(0, x), 10000)

  1.196768 seconds (828.74 k allocations: 32.672 MiB, 3.49% gc time, 0.43% compilation time)


In [36]:
Time_benchmark(x -> BS_Cos_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.049413 seconds (205.39 k allocations: 9.699 MiB, 27.65% compilation time)


---

In [15]:
Funcs_Compare(x -> BS_Cos_Sys_Simp(zeros(6, 1), x, [K, a2, a4, r, Eps], 0),
              x -> BS_Cos_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

1.1190321672766013e-16

In [45]:
Time_benchmark(x -> BS_Cos_Sys_Simp(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.054919 seconds (205.39 k allocations: 9.699 MiB, 26.65% compilation time)


In [46]:
Time_benchmark(x -> BS_Cos_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0), 10000)

  0.053657 seconds (205.39 k allocations: 9.699 MiB, 25.94% compilation time)


---

# Сравнение разницы якобианов посчитанных при помощи ForwardDiff и явно заданных

In [22]:
Funcs_Compare(z -> ForwardDiff.jacobian((y, x) -> BS_Sin_Sys(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z),
              z -> BS_Sin_Jac(z, [K, a2, a4, r, Eps], 0))

3.774758283725532e-15

In [49]:
Time_benchmark(z -> ForwardDiff.jacobian((y, x) -> BS_Sin_Sys(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z), 10000)

  7.927533 seconds (28.88 M allocations: 3.154 GiB, 5.48% gc time, 12.86% compilation time)


In [50]:
Time_benchmark(z -> BS_Sin_Jac(z, [K, a2, a4, r, Eps], 0), 10000)

  7.195040 seconds (73.00 M allocations: 3.755 GiB, 8.40% gc time, 0.06% compilation time)


---

In [24]:
Funcs_Compare(z -> ForwardDiff.jacobian((y, x) -> BS_Sin_Sys_Simp(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z),
              z -> BS_Sin_Jac(z, [K, a2, a4, r, Eps], 0))

1.6653345369377348e-16

In [51]:
Time_benchmark(z -> ForwardDiff.jacobian((y, x) -> BS_Sin_Sys_Simp(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z), 10000)

  1.102547 seconds (3.96 M allocations: 234.212 MiB, 4.29% gc time, 94.45% compilation time)


In [52]:
Time_benchmark(z -> BS_Sin_Jac(z, [K, a2, a4, r, Eps], 0), 10000)

  0.066147 seconds (728.68 k allocations: 38.468 MiB, 6.41% compilation time)


---

In [23]:
Funcs_Compare(z -> ForwardDiff.jacobian((y, x) -> BS_Cos_Sys(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z),
              z -> BS_Cos_Jac(z, [K, a2, a4, r, Eps], 0))

3.774758283725532e-15

In [53]:
Time_benchmark(z -> ForwardDiff.jacobian((y, x) -> BS_Cos_Sys(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z), 10000)

  1.144342 seconds (4.15 M allocations: 239.986 MiB, 6.41% gc time, 92.49% compilation time)


In [54]:
Time_benchmark(z -> BS_Cos_Jac(z, [K, a2, a4, r, Eps], 0), 10000)

  0.063972 seconds (728.68 k allocations: 38.468 MiB, 6.92% compilation time)


---

In [25]:
Funcs_Compare(z -> ForwardDiff.jacobian((y, x) -> BS_Cos_Sys_Simp(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z),
              z -> BS_Cos_Jac(z, [K, a2, a4, r, Eps], 0))

1.734723475976807e-16

In [59]:
Time_benchmark(z -> ForwardDiff.jacobian((y, x) -> BS_Cos_Sys_Simp(y, x, [K, a2, a4, r, Eps], 0), zeros(6, 1), z), 10000)

  1.112437 seconds (3.97 M allocations: 234.690 MiB, 4.15% gc time, 94.45% compilation time)


In [60]:
Time_benchmark(z -> BS_Cos_Jac(z, [K, a2, a4, r, Eps], 0), 10000)

  0.064997 seconds (728.68 k allocations: 38.468 MiB, 6.67% compilation time)


# Сравнение времени расчета спектра ляпуновских показателей

In [102]:
LE_Time = 10000
dt = 0.01
Ttr = 1000
alg = DP8()
abstol = 1e-13
reltol = 1e-13
maxiters = 1e10

1.0e10

---

In [87]:
lvDs = ContinuousDynamicalSystem(BS_Cos_Sys, startPt, [K, a2, a4, r, Eps], BS_Cos_Jac)

6-dimensional continuous dynamical system
 state:       [3.14149, 0.0001, 0.0001, 3.14149, 0.0001, 3.14149]
 rule f:      BS_Cos_Sys
 in-place?    true
 jacobian:    BS_Cos_Jac
 parameters:  [0.4, 1.5708, 3.14159, 0.01, 1.0e-10]

In [103]:
@time lyapunovspectrum(lvDs, LE_Time; diffeq = (dt = dt, Ttr = Ttr, alg = alg, abstol = abstol, reltol = reltol, maxiters = maxiters), show_progress=true)

Lyapunov Spectrum: 100%|████████████████████████████████| Time: 0:00:02


  2.827248 seconds (26.80 M allocations: 1.352 GiB, 6.25% gc time)


6-element Vector{Float64}:
  0.0006239405535510849
 -0.0020344412752627337
 -0.03792152690844226
 -0.053159483008218406
 -0.05817444601591517
 -0.07543283310315227

---

In [104]:
lvDs = ContinuousDynamicalSystem(BS_Cos_Sys, startPt, [K, a2, a4, r, Eps])

6-dimensional continuous dynamical system
 state:       [3.14149, 0.0001, 0.0001, 3.14149, 0.0001, 3.14149]
 rule f:      BS_Cos_Sys
 in-place?    true
 jacobian:    ForwardDiff
 parameters:  [0.4, 1.5708, 3.14159, 0.01, 1.0e-10]

In [107]:
@time lyapunovspectrum(lvDs, LE_Time; diffeq = (dt = dt, Ttr = Ttr, alg = alg, abstol = abstol, reltol = reltol, maxiters = maxiters), show_progress=true)

Lyapunov Spectrum: 100%|████████████████████████████████| Time: 0:00:01


  1.201867 seconds (40.51 k allocations: 14.096 MiB, 2.51% gc time)


6-element Vector{Float64}:
  0.0006413027167435878
 -0.0020828673160153068
 -0.03798251014290281
 -0.052953089219596905
 -0.05826958433285318
 -0.075452041462816

---

In [17]:
Funcs_Compare(x -> BS_Exp_Sys(zeros(6, 1), x, [K, a2, a4, r, Eps], 0),
              x -> BS_Uni_Sys(couple_exp)(zeros(6, 1), x, [K, a2, a4, r, Eps], 0))

6.217248937900877e-15